<a href="https://colab.research.google.com/github/wajnryt/JerusMLDeepLearning2019/blob/master/youtube_bb_datacreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install youtube-dl imageio moviepy ffmpy Pillow opencv-python


     |████████████████████████████████| 1.8MB 4.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/16/28/57/96aff0d874198125b03f542d854e7ebdc61a56b09a4d49de6a
Successfully built ffmpy


In [0]:
from google.colab import drive
import os
drive.mount('/content/gdrive')

dl_dir = '/content/gdrive/My Drive/videos_2'
if not os.path.exists(dl_dir):
  os.mkdir(dl_dir)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import youtube_dl
import os
debug = False

d_sets = [
          'yt_bb_detection_train',
          'yt_bb_detection_validation',
          'yt_bb_classification_train',
          'yt_bb_classification_validation',
          ]

# The classes included and their indices
class_list = [\
              [0,'person'],
              [1,'bird'],
              [2,'bicycle'],
              [3,'boat'],
              [4,'bus'],
              [5,'bear'],
              [6,'cow'],
              [7,'cat'],
              [8,'giraffe'],
              [9,'potted plant'],
              [10,'horse'],
              [11,'motorcycle'],
              [12,'knife'],
              [13,'airplane'],
              [14,'skateboard'],
              [15,'train'],
              [16,'truck'],
              [17,'zebra'],
              [18,'toilet'],
              [19,'dog'],
              [20,'elephant'],
              [21,'umbrella'],
              [22,'none'],
              [23,'car'],
              ]

web_host = 'https://research.google.com/youtube-bb/'

In [0]:
import pandas as pd
from subprocess import check_call
d_set = 'yt_bb_detection_train' # for d_set in in youtube_bb.d_sets:
d_set_dir = dl_dir+'/'+d_set+'/'

# Download & extract the annotation list
if not os.path.exists(d_set+'.csv'):
  print (d_set+': Downloading annotations...')
  check_call(' '.join(['wget', web_host+d_set+'.csv.gz']),shell=True)
  print (d_set+': Unzipping annotations...')
  check_call(' '.join(['gzip', '-d', '-f', d_set+'.csv.gz']), shell=True)    

data = pd.read_csv(d_set+'.csv', 
                   #header=None,
                   names = 'youtube_id timestamp_ms class_id class_name object_id object_presence xmin xmax ymin ymax'.split())
data = data[data['object_presence']=='present']
print(len(data))
print(data.head(5))

yt_bb_detection_train: Downloading annotations...
yt_bb_detection_train: Unzipping annotations...
4484018
    youtube_id  timestamp_ms  class_id  ...   xmax      ymin      ymax
0  AAB6lO-XiKE        238000         0  ...  0.540  0.371667  0.616667
1  AAB6lO-XiKE        239000         0  ...  0.588  0.363333  0.606667
2  AAB6lO-XiKE        240000         0  ...  0.614  0.443333  0.685000
3  AAB6lO-XiKE        241000         0  ...  0.605  0.448333  0.686667
4  AAB6lO-XiKE        242000         0  ...  0.600  0.400000  0.638333

[5 rows x 10 columns]


In [0]:
import imageio
import subprocess
from subprocess import check_call

def parse_vid(yt_id, vid_path):
  image_dat = data[data['youtube_id']==yt_id]
  reader  = imageio.get_reader(vid_path,  'ffmpeg')
  fps = reader.get_meta_data()['fps']
  for index, im_dat in image_dat.iterrows():
    decode_time = im_dat['timestamp_ms']
    image_path = os.path.join(d_set_dir, str(im_dat['class_id']), yt_id+'_'+str(im_dat['object_id']))
    image_file = os.path.join(image_path, f'{decode_time}.jpg')

    if os.path.exists(image_file):
      continue
    if not os.path.exists(image_path):
      os.makedirs(image_path)

    image_id = int(decode_time/1000*fps)
    im = reader.get_data(image_id)
    pix_x = (im_dat[['xmin', 'xmax']]*im.shape[1]).astype(int)
    pix_y = (im_dat[['ymin', 'ymax']]*im.shape[0]).astype(int)

    crop = im[pix_y[0]:pix_y[1], pix_x[0]:pix_x[1]]
    imageio.imwrite(image_file, crop) #consider growing the crop by 50%


In [0]:
import numpy as np
video_ids = np.unique(data['youtube_id'])
print('num unique videos = ', len(video_ids))
for yt_id in video_ids:
  target_full = os.path.join(d_set_dir, yt_id+'_full.mp4')
  if os.path.exists(target_full):
    continue
  cmd = ['youtube-dl', 
         '-f', 'best[ext=mp4]', 
         '-o',target_full, 
         'youtu.be/'+yt_id ]
  print(' '.join(cmd))
  try:
    check_call(cmd, stderr=subprocess.STDOUT)
    parse_vid(yt_id, target_full)
  except:
    print('something failed with ', yt_id)

num unique videos =  188546
youtube-dl -f best[ext=mp4] -o /content/gdrive/My Drive/videos_2/yt_bb_detection_train/--IABqeR5W8_full.mp4 youtu.be/--IABqeR5W8
something failed with  --IABqeR5W8
youtube-dl -f best[ext=mp4] -o /content/gdrive/My Drive/videos_2/yt_bb_detection_train/--VxF9iwB14_full.mp4 youtu.be/--VxF9iwB14
something failed with  --VxF9iwB14
youtube-dl -f best[ext=mp4] -o /content/gdrive/My Drive/videos_2/yt_bb_detection_train/--XBA213pXY_full.mp4 youtu.be/--XBA213pXY
something failed with  --XBA213pXY
youtube-dl -f best[ext=mp4] -o /content/gdrive/My Drive/videos_2/yt_bb_detection_train/--loEWg4p58_full.mp4 youtu.be/--loEWg4p58
something failed with  --loEWg4p58
youtube-dl -f best[ext=mp4] -o /content/gdrive/My Drive/videos_2/yt_bb_detection_train/-0Xb0BmX-ss_full.mp4 youtu.be/-0Xb0BmX-ss
something failed with  -0Xb0BmX-ss
youtube-dl -f best[ext=mp4] -o /content/gdrive/My Drive/videos_2/yt_bb_detection_train/-0Y8h8Jrozk_full.mp4 youtu.be/-0Y8h8Jrozk
something failed with  